In [2]:
PATH = "/home/xchen/GANs/WGAN-div-PyTorch-/samples/1204_mnist_matsumoto_1/"
FILE_NAME = "1204_mnist_matsumoto_1"

In [11]:
import os

list = os.listdir(PATH)
list.sort(key=lambda fn: os.path.getmtime(PATH+fn) if not os.path.isdir(PATH+fn) else 0)
print(list[-1])

9900


In [24]:
def fid_all_list(filePath, fid_dict={}):
    fid = 'python3 -m pytorch_fid'

    for i in os.listdir(filePath):
        PATH_with_FILE = os.path.join(filePath, i)
        FILE_NAME = i

        file_list = os.listdir(PATH_with_FILE)
        file_list.sort(key=lambda fn: os.path.getmtime(os.path.join(PATH_with_FILE,fn)) if not os.path.isdir(os.path.join(PATH_with_FILE,fn)) else 0)
        max_value = int(file_list[-1]) + 1

        # start FID calc
        for i in range(0, max_value, 100):

            real_path = ' ' + os.path.join(PATH_with_FILE, str(i)) + '/real_images'
            fake_path = ' ' + os.path.join(PATH_with_FILE, str(i)) + '/fake_images'

            command_line = fid + real_path + fake_path

            args = shlex.split(command_line)

            res = subprocess.run(args, shell=False, stdout=subprocess.PIPE, text=True)

            fid_dict[i] = float(res.stdout[6:-1])

    with open(FILE_NAME +'.log', "w") as tf:

        print(PATH_with_FILE + '\n', file=tf)

        pprint.pprint(sorted(fid_dict.items(), key=lambda kv:kv[1]), stream=tf)
            

In [32]:
def fid_one_list(filePath, fileNAME, fid_dict={}):
    fid = 'python3 -m pytorch_fid'

    PATH_with_FILE = os.path.join(filePath, fileNAME)

    file_list = os.listdir(PATH_with_FILE)
    file_list.sort(key=lambda fn: os.path.getmtime(os.path.join(PATH_with_FILE,fn)) if not os.path.isdir(os.path.join(PATH_with_FILE,fn)) else 0)
    max_value = int(file_list[-1]) + 1

    # start FID calc 
    for i in range(0, max_value, 100):

        real_path = ' ' + os.path.join(PATH_with_FILE, str(i)) + '/real_images'
        fake_path = ' ' + os.path.join(PATH_with_FILE, str(i)) + '/fake_images'

        command_line = fid + real_path + fake_path

        # split command line 
        args = shlex.split(command_line)

        res = subprocess.run(args, shell=False, stdout=subprocess.PIPE, text=True)

        fid_dict[i] = float(res.stdout[6:-1])

    with open(fileNAME + '.log', 'w') as tf:

        print(PATH_with_FILE + '\n', file=tf)

        pprint.pprint(sorted(fid_dict.items(), key=lambda kv:kv[1]), stream=tf)

In [33]:
import os 
import pprint
import shlex
import subprocess

filePath = "/home/xchen/GANs/WGAN-div-PyTorch-/samples/"
fileNAME = "1204_mnist_matsumoto_1"

# fid_all_list(filePath)
fid_one_list(filePath, fileNAME)


  0%|          | 0/202 [00:00<?, ?it/s]/home/xchen/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
  0%|          | 0/202 [00:00<?, ?it/s]/home/xchen/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
  0%|          | 0/202 [00:00<?, ?it/s]/home/xchen/anaconda3/lib/python3.7/site-packages/tor

In [3]:
import sys 
sys.path.append('..')
import torch 
from models.wgan_div import Generator

ckpt_path = "/home/xchen/GANs/WGAN-div-PyTorch-/checkpoint/1216_cifar10_bs64_matsumoto_1/1500.pth.tar"

G = Generator().cuda()
fixed_noise = torch.randn(10000, 100).cuda()

torch.load(ckpt_path)

fake_images = G(fixed_noise)

FileNotFoundError: [Errno 2] No such file or directory: '/home/xchen/GANs/WGAN-div-PyTorch-/checkpoint/1216_cifar10_bs64_matsumoto_1/1500.pth.tar'

In [14]:
import sys 
sys.path.append('..')
from models.wgan_div import Generator, Discriminator
from torchinfo import summary
import pprint

G = Generator().cuda()
D = Discriminator().cuda()

# get model summary as string 
G_model_stats = summary(G, (64, 100), verbose=0)
D_model_stats = summary(D, (64, 1, 64, 64), verbose=0)

pprint.pprint(G_model_stats)
pprint.pprint(D_model_stats)
summary_str = str(G_model_stats), str(D_model_stats)
with open('model_structure.log', 'w') as tf:
    pprint.pprint(summary_str, stream=tf)

Layer (type:depth-idx)                   Output Shape              Param #
Generator                                --                        --
├─Sequential: 1-1                        [64, 512, 4, 4]           --
│    └─ConvTranspose2d: 2-1              [64, 512, 4, 4]           819,200
│    └─BatchNorm2d: 2-2                  [64, 512, 4, 4]           1,024
│    └─ReLU: 2-3                         [64, 512, 4, 4]           --
├─Sequential: 1-2                        [64, 256, 8, 8]           --
│    └─ConvTranspose2d: 2-4              [64, 256, 8, 8]           2,097,152
│    └─BatchNorm2d: 2-5                  [64, 256, 8, 8]           512
│    └─ReLU: 2-6                         [64, 256, 8, 8]           --
├─Sequential: 1-3                        [64, 128, 16, 16]         --
│    └─ConvTranspose2d: 2-7              [64, 128, 16, 16]         524,288
│    └─BatchNorm2d: 2-8                  [64, 128, 16, 16]         256
│    └─ReLU: 2-9                         [64, 128, 16, 16]     